In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import logistic, LogisticRegression as LR
import os, sys
import pickle
from tqdm import tqdm
import notebook_utils as nbutils
import data_utils as datutil
import datetime as dt
import hmc
from models import *
import gpytorch
from notebook_utils import *
from sklearn.preprocessing import LabelBinarizer as lab_biner

In [2]:
# Data loader initialization
trainloader = datutil.generate_dataloaders('ENCODED256_D164_CIFAR10_TRAIN', batch_size=50000, shuffle=False, num_workers=2)
testloader = datutil.generate_dataloaders('ENCODED256_D164_CIFAR10_TEST', batch_size=10000, shuffle=False, num_workers=2)

trainX, trainY = iter(trainloader).next()
testX, testY = iter(testloader).next()
trainX, trainY = trainX.cpu().numpy(), trainY.cpu().numpy()
testX, testY = testX.cpu().numpy(), testY.cpu().numpy()

num_class = 10
num_feature = 256
total_param_size = num_class * (num_feature + 1)

# loss function needed for when direct bfgs method is 
# used to reduce loss and return hessian matrix
def loss_fn(x):
    
    print(trainY.shape)
    w_size = num_feature*num_class
    w = x[:w_size].reshape((num_feature, num_class))
    b = x[w_size:]
    class_score = trainX.dot(w) + b
    exp_class_score = np.exp(class_score)
    sum_exp_class_score = np.tile(np.sum(exp_class_score, axis=1).reshape((-1, 1)), (1,10))
    softmax = (exp_class_score / sum_exp_class_score)
    loss = -np.log(softmax[:,trainY]).sum()
    return loss

In [3]:
# sklearn logistic regression fitting
logisticReg = LR(penalty='l2', solver='lbfgs', multi_class='multinomial', C=1., max_iter=1000)
logisticReg.fit(trainX, trainY)
print('Train score is', logisticReg.score(trainX, trainY))
print('Test score is', logisticReg.score(testX, testY))

Train score is 1.0
Test score is 0.9213


In [4]:
# parameter at which hessian needs to be computed i.e. the fitted coef and intercept
param = np.concatenate((logisticReg.coef_, logisticReg.intercept_.reshape(-1, 1)), axis=1)
oneHoter = lab_biner()
oneHoter.fit(trainY)
train1hot = oneHoter.transform(trainY)
test1hot = oneHoter.transform(testY)
# gradient at the fitted coefs and hessian.p method
grad, hess_fn = logistic._multinomial_grad_hess(param, trainX, train1hot, alpha=1., sample_weight=np.ones(len(trainX)))

In [5]:
# procedure to capture diagonal entries of the hessian matrix
# calls the hessian.p method number of parameter times, to populate
# the hessian matrix diagonal values
hess_diag = np.zeros(param.shape[0]*param.shape[1])

for i in tqdm(range(param.shape[0]*param.shape[1])):
    vector = np.zeros(param.shape[0]*param.shape[1])
    vector[i] = 1
    vector = vector.reshape(param.shape)
    hess_diag[i] = hess_fn(vector)[i]
    
with open('mcmc-samples_hessians/CIFAR10_Res164_hess.pkl', 'wb') as openf:
    pickle.dump(hess_diag, openf)

  1%|          | 19/2570 [00:03<07:54,  5.37it/s]


KeyboardInterrupt: 

In [6]:
with open('mcmc-samples_hessians/CIFAR10_Res164_hess.pkl', 'rb') as openf:
    hess_diag = pickle.load(openf)

In [ ]:
batch_count = 0
correct = 0
total = 0
# inverse of diagonal of square root of Hessian
sigma = 1. / np.sqrt(hess_diag.reshape((num_class, num_feature+1)))
w_sigma = sigma[:, :num_feature]
b_sigma = sigma[:, num_feature].reshape(num_class)
outputs = 0
labels = torch.tensor(testY)
pred_list, prob_list, target_list = [], [], []
total_sample = 100

'''
Generate samples from laplace approximation and then compute probabilities
from each of the samples and then averages the class probabilities across samples
'''
for sample_count in range(total_sample):
    w = logisticReg.coef_ + w_sigma * np.random.normal(size=(logisticReg.coef_.shape))
    b = logisticReg.intercept_ + b_sigma * np.random.normal(size=(logisticReg.intercept_.shape))
    outputs += F.softmax(torch.tensor((w.dot(testX.T)).T + b), dim=1) / total_sample

max_prob, predicted = torch.max(outputs, 1)
c = (predicted == labels).squeeze()
correct += c.sum().item()
total += labels.size(0)

for prob, label, pred in zip(max_prob, labels, predicted):
    pred_list.append(pred.item())
    target_list.append(label.item())
    prob_list.append(prob.item())

acc = 100.0 * correct / total
print("Accuracy statistics")
print('Overall accuracy : %2d %%' % (acc))

bins = [(p + 1) / 30.0 for p in range(30)]
ece_mid, ece_avg = nbutils.calculate_ECE(probs=prob_list, preds=pred_list, targets=target_list, ECE_bin=bins)
print('ECE values are %.3f, %.3f when mid bin and avg used respectively' % (ece_mid, ece_avg))